In [130]:
from pyaxis import pyaxis
import pandas as pd

#set file path (or URL)
fp = r"/Users/boasnydegger/Desktop/github/dsfgp/data/px-x-1003020000_101.px" 
#parse contents of *.px file
px = pyaxis.parse(uri = fp , encoding = 'ISO-8859-2')

#store data as pandas dataframe
df = px['DATA']
meta_dict = px['METADATA']
#store metadata as a dictionary (not necessary in this case, but might be helpful)

print("Alex")
print("Test Boas")
print("bla")

Alex
Test Boas
bla


In [140]:
df = df.loc[df["Indikator"] == "Logiernächte"]
df['DATA'] = df['DATA'].str.replace('"..."','0')
df['DATA'] = df['DATA'].str.replace('"......"','0')

/var/folders/pj/5_51d77x0lbdcvsbw8bfblkc0000gn/T/ipykernel_89460/2157644305.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['DATA'] = df['DATA'].str.replace('"..."','0')
/var/folders/pj/5_51d77x0lbdcvsbw8bfblkc0000gn/T/ipykernel_89460/2157644305.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['DATA'] = df['DATA'].str.replace('"......"','0')


In [141]:
df["Jahr"] = df["Jahr"].astype(int)
df["DATA"] = df["DATA"].astype(int)

In [142]:
df

,Jahr,Monat,Gemeinde,Herkunftsland,Indikator,DATA
1,2013,Jahrestotal,Kloten,Herkunftsland - Total,Logiernächte,237998
3,2013,Jahrestotal,Kloten,Schweiz,Logiernächte,70932
5,2013,Jahrestotal,Kloten,Deutschland,Logiernächte,21688
7,2013,Jahrestotal,Kloten,Frankreich,Logiernächte,5858
9,2013,Jahrestotal,Kloten,Italien,Logiernächte,4801
...,...,...,...,...,...,...
1935951,2022,Dezember,Meyrin,Slowakei,Logiernächte,0
1935953,2022,Dezember,Meyrin,Slowenien,Logiernächte,0
1935955,2022,Dezember,Meyrin,Tschechien,Logiernächte,0
1935957,2022,Dezember,Meyrin,Ukraine,Logiernächte,0


In [100]:
saanen=df.loc[(df["Gemeinde"]=="Saanen")&(df["Indikator"]=="Logiernächte")]
stmoritz=df.loc[(df["Gemeinde"]=="St. Moritz")&(df["Indikator"]=="Logiernächte")]
andermatt=df.loc[(df["Gemeinde"]=="Andermatt")&(df["Indikator"]=="Logiernächte")]
#saanen=df.loc[(df["Gemeinde"]=="Saanen")&(df["Indikator"]=="Logiernächte")]

In [110]:
saanen.loc[(saanen["Monat"] == "September")&(saanen["Herkunftsland"]== "Herkunftsland - Total")]

,Jahr,Monat,Gemeinde,Herkunftsland,Indikator,DATA
136949,2013,September,Saanen,Herkunftsland - Total,Logiernächte,23558
330545,2014,September,Saanen,Herkunftsland - Total,Logiernächte,24808
524141,2015,September,Saanen,Herkunftsland - Total,Logiernächte,25580
717737,2016,September,Saanen,Herkunftsland - Total,Logiernächte,22449
911333,2017,September,Saanen,Herkunftsland - Total,Logiernächte,26599
1104929,2018,September,Saanen,Herkunftsland - Total,Logiernächte,27771
1298525,2019,September,Saanen,Herkunftsland - Total,Logiernächte,29051
1492121,2020,September,Saanen,Herkunftsland - Total,Logiernächte,27849
1685717,2021,September,Saanen,Herkunftsland - Total,Logiernächte,34396
1879313,2022,September,Saanen,Herkunftsland - Total,Logiernächte,30397


In [112]:
df = df.loc[df["Indikator"] == "Logiernächte"]

 

df = df.drop(df.loc[df["Monat"]=="Jahrestotal"], inplace=True)
#df = df.drop((df.loc[(df["Jahr"]== 2022)&(df["Monat"]=="Dezember")]))

KeyError: 'Indikator'

In [109]:
#df1 = df.drop(df.loc[(df["Jahr"]=="2022")&(df["Monat"]=="Dezember")])
df

1          2013.0
3          2013.0
5          2013.0
7          2013.0
9          2013.0
            ...  
1935951    2022.0
1935953    2022.0
1935955    2022.0
1935957    2022.0
1935959    2022.0
Name: Jahr, Length: 967980, dtype: float64

In [90]:
df

,Jahr,Monat,Gemeinde,Herkunftsland,Indikator,DATA
1,2013,Jahrestotal,Kloten,Herkunftsland - Total,Logiernächte,237998
3,2013,Jahrestotal,Kloten,Schweiz,Logiernächte,70932
5,2013,Jahrestotal,Kloten,Deutschland,Logiernächte,21688
7,2013,Jahrestotal,Kloten,Frankreich,Logiernächte,5858
9,2013,Jahrestotal,Kloten,Italien,Logiernächte,4801
...,...,...,...,...,...,...
1742355,2021,Dezember,Meyrin,Slowakei,Logiernächte,10
1742357,2021,Dezember,Meyrin,Slowenien,Logiernächte,0
1742359,2021,Dezember,Meyrin,Tschechien,Logiernächte,34
1742361,2021,Dezember,Meyrin,Ukraine,Logiernächte,20


In [47]:

df

,Jahr,Monat,Gemeinde,Herkunftsland,Indikator,DATA
0,2013,Jahrestotal,Kloten,Herkunftsland - Total,Ankünfte,177829
1,2013,Jahrestotal,Kloten,Herkunftsland - Total,Logiernächte,237998
2,2013,Jahrestotal,Kloten,Schweiz,Ankünfte,59437
3,2013,Jahrestotal,Kloten,Schweiz,Logiernächte,70932
4,2013,Jahrestotal,Kloten,Deutschland,Ankünfte,14447
...,...,...,...,...,...,...
1935955,2022,Dezember,Meyrin,Tschechien,Logiernächte,0
1935956,2022,Dezember,Meyrin,Ukraine,Ankünfte,0
1935957,2022,Dezember,Meyrin,Ukraine,Logiernächte,0
1935958,2022,Dezember,Meyrin,Serbien,Ankünfte,0


In [82]:
df2 = df

In [86]:
df2 = df2[df2.Jahr != "2022"]
df2

,Jahr,Monat,Gemeinde,Herkunftsland,Indikator,DATA
1,2013,Jahrestotal,Kloten,Herkunftsland - Total,Logiernächte,237998
3,2013,Jahrestotal,Kloten,Schweiz,Logiernächte,70932
5,2013,Jahrestotal,Kloten,Deutschland,Logiernächte,21688
7,2013,Jahrestotal,Kloten,Frankreich,Logiernächte,5858
9,2013,Jahrestotal,Kloten,Italien,Logiernächte,4801
...,...,...,...,...,...,...
1742355,2021,Dezember,Meyrin,Slowakei,Logiernächte,10
1742357,2021,Dezember,Meyrin,Slowenien,Logiernächte,0
1742359,2021,Dezember,Meyrin,Tschechien,Logiernächte,34
1742361,2021,Dezember,Meyrin,Ukraine,Logiernächte,20


In [81]:
df2.drop(df2.loc[df2['Jahr']=="2022"].index, inplace=True)

/var/folders/pj/5_51d77x0lbdcvsbw8bfblkc0000gn/T/ipykernel_89460/1836912238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(df2.loc[df2['Jahr']=="2022"].index, inplace=True)


In [67]:
df2

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 871182 entries, 1 to 1742363
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Jahr           871182 non-null  object
 1   Monat          871182 non-null  object
 2   Gemeinde       871182 non-null  object
 3   Herkunftsland  871182 non-null  object
 4   Indikator      871182 non-null  object
 5   DATA           871182 non-null  object
dtypes: object(6)
memory usage: 46.5+ MB
